In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
metadf = pd.read_csv("cleaned_metadf.csv")

In [ ]:
X = metadf.drop('algorithm',axis=1)
y = metadf['algorithm']

In [ ]:
import numpy as np
from scipy.stats import mode
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming 'X' is your feature matrix and 'y' is the target variable
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

# Initialize the individual models with random state 33
rf_model = RandomForestClassifier(n_estimators=10, max_depth=11, random_state=42)
ada_model = AdaBoostClassifier(n_estimators=100, random_state=33)
gbm_model = GradientBoostingClassifier(n_estimators=100, random_state=33)

# Fit the individual models on the training data
rf_model.fit(X_train, y_train)
ada_model.fit(X_train, y_train)
gbm_model.fit(X_train, y_train)

# Make predictions with each model on the testing data
rf_pred = rf_model.predict(X_test)
ada_pred = ada_model.predict(X_test)
gbm_pred = gbm_model.predict(X_test)

# Create an ensemble by taking the mode (most frequent class) of the predictions from the three models
ensemble_pred = np.array([rf_pred, ada_pred, gbm_pred])
final_predictions = mode(ensemble_pred, axis=0).mode.flatten()

# Calculate the accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)

# Display the accuracy of each individual model and the ensemble
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("AdaBoost Accuracy:", accuracy_score(y_test, ada_pred))
print("Gradient Boosting Accuracy:", accuracy_score(y_test, gbm_pred))
print("Ensemble Accuracy:", ensemble_accuracy)

# You can also print other evaluation metrics such as classification report or confusion matrix
print("\nEnsemble Classification Report:")
print(classification_report(y_test, final_predictions))

print("\nEnsemble Confusion Matrix:")
print(confusion_matrix(y_test, final_predictions))


In [41]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score

def cross_val_with_oversampling(classifier, X, y, n_splits=5, random_state=42):
    # Convert X and y to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Perform stratified k-fold cross-validation
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # Initialize the oversampler
    oversampler = RandomOverSampler(random_state=random_state)

    # Initialize lists to store the accuracy scores for each fold
    cv_scores = []

    # Perform cross-validation with class oversampling
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Perform class oversampling on the training set
        X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

        # Fit the classifier on the resampled data
        classifier.fit(X_train_resampled, y_train_resampled)

        # Make predictions on the test set
        y_pred = classifier.predict(X_test)

        # Calculate accuracy for this fold
        fold_accuracy = accuracy_score(y_test, y_pred)
        cv_scores.append(fold_accuracy)

    # Return the cross-validation accuracy scores
    return cv_scores

# Example usage with RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Assuming you have already loaded your dataset into X (features) and y (labels) arrays
random_state = 42

rf_classifier = RandomForestClassifier(n_estimators=10, max_depth=11, random_state=random_state)

cv_scores_rf = cross_val_with_oversampling(rf_classifier, X, y, random_state=random_state)

print("Cross-validation Accuracy Scores (Random Forest): ", cv_scores_rf)
print("Average Accuracy (Random Forest): ", np.mean(cv_scores_rf))


Cross-validation Accuracy Scores (Random Forest):  [0.6470588235294118, 0.5882352941176471, 0.5625, 0.625, 0.4375]
Average Accuracy (Random Forest):  0.5720588235294117
